In [1]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


Game = Regency()
clear_output()
for a in range(1):
    Game.random_events()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_regency_points()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.domain_initiative()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.collect_gold_bars()
    print(len((Game.Seasons[Game.Season]['Season'])))
    Game.maintenance_costs()
    print(len((Game.Seasons[Game.Season]['Season'])))

Game.clear_screen()
Game.Seasons[Game.Season]['Season']

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,Ch,False,Natural Event,28,24,0,0,0,Dormant
1,GG,False,No Event,77,21,52,42,30,Rich
2,Sw2,False,Unrest or Rebellion,24,20,1,0,2,Bare
3,Is,False,No Event,106,19,7,14,7,Bare
4,WB,False,No Event,2,16,1,2,1,Dormant
5,LPA,False,No Event,11,13,7,11,4,Bare
6,ML,False,No Event,3,12,1,2,1,Dormant
7,CoL,False,No Event,27,12,0,6,6,Bare
8,IHH,False,No Event,6,10,1,4,3,Bare
9,EC,False,Monsters,33,10,-1,11,12,Dormant


In [2]:
Game.take_domain_actions(None)

UnboundLocalError: local variable 'name' referenced before assignment

In [ ]:
Game.Troops

In [ ]:
Game.Regents

In [ ]:
Game.Projects

In [ ]:
self =  Game
Type = 'Archers'

temp = self.troop_units[self.troop_units['Unit Type'] == Type]['Maintenance Cost'].values[0]

temp